In [1]:

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import metrics #метрики


In [2]:
df = sns.load_dataset('diamonds')

In [3]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [4]:
df = pd.get_dummies(df, drop_first=True)

In [5]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [6]:
X = df.drop(columns="price")
y = df["price"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, #таблица с наблюдениями и столбец с правильным ответами
    test_size=0.33, #размер тестовой выборки
    random_state=42 #число отвечающее за генерацию случайных чисел
)

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

param_grid = {"loss": ["squared_error", "epsilon_insensitive"],
            "penalty": ["elasticnet"],
            "alpha": np.logspace(-3, 3, 15),
            "l1_ratio": np.linspace(0, 1, 11),
            "max_iter": np.logspace(0, 3, 10).astype(int),
            "random_state": [42],
            "learning_rate": ["constant"],
            "eta0": np.logspace(-4, -1, 4)}
grid_search = GridSearchCV(
    estimator=SGDRegressor(), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
print('Train MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_test_pred)))

Наилучшие значения гиперпараметров: {'alpha': 0.0026826957952797246, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'max_iter': 21, 'penalty': 'elasticnet', 'random_state': 42}
Train MAE: 0.159


In [9]:
metrics.mean_squared_error(y_test, y_test_pred)

0.04349853776551418